In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
! pip install transformers

     |████████████████████████████████| 1.9MB 5.5MB/s 
     |████████████████████████████████| 3.2MB 20.1MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a21bab1ce7aea236b191a36402929b97a97853491ccc6f0d828ff25a82998481
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import transformers as ppb
from ast import literal_eval

In [4]:
diagnosis_icd9category=pd.read_csv("/content/drive/MyDrive/AlgoIntern/Data/Data2.0/diagnosis_icd9category.csv")
# diagnosis_icd9category=pd.read_csv("/content/drive/My Drive/Mrityunjay Samanta (Intern Work)/diagnosis_icd9category.csv")

In [5]:
diagnosis_icd9category.dropna(subset=["discharge diagnosis:"],inplace=True)
diagnosis_icd9category['icd9_category']=diagnosis_icd9category.icd9_category.apply(literal_eval)
diagnosis_icd9category['medical_terms_list']=diagnosis_icd9category.medical_terms_list.apply(literal_eval)

In [6]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list
2,167118.0,"discharge diagnosis: copd, coronary artery di...","[496, 553, 518, 453, 519, 276]","[coronary artery disease/atypical angina, hype..."
3,196489.0,discharge diagnosis: primary: 1. chronic obs...,"[487, 416, 272, 530, 414, 553, 531, 518, 401, ...","[chronic obstructive pulmonary disease, respir..."
4,135453.0,discharge diagnosis: 1. cervical spondylosis ...,"[V12, 788, 805, 507, V45, 401, 427, 453, E88, ...","[spondylosis, calcification of posterior, frac..."
5,170490.0,discharge diagnosis: brain lesion discharg...,"[710, 225, 443, 714, 530, V43]","[infection, pain, pain, constipation, docusate..."
6,134727.0,discharge diagnosis: pneumonia discharge c...,"[486, 518, V45, V58, 401, 577, 415, 428]",[]
...,...,...,...,...
59606,127716.0,discharge diagnosis: aortic stenosis and coro...,"[416, 424, 272, V10, V85, 414, 250, 401, 278, ...","[coronary artery disease, erythema, edema, red..."
59609,186377.0,discharge diagnosis: coronary artery disease ...,"[424, 511, 305, 707, 403, 496, 784, 250, 414, ...","[diabetes, diastolic heart failure, erythema, ..."
59615,156564.0,discharge diagnosis: left knee infection chr...,"[V10, 403, 584, V43, 428, 285, 496, 250, V58, ...","[obstructive pulmonary disease, kidney injury,..."
59631,172559.0,discharge diagnosis: primary: small bowel ob...,"[197, 507, 185, 427, 261, 585, V66, 560, 783, ...","[bowel obstruction, prostate cancer, diverticu..."


In [7]:
diagnosis_icd9category.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41967 entries, 2 to 59641
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HADM_ID               41967 non-null  float64
 1   discharge diagnosis:  41967 non-null  object 
 2   icd9_category         41967 non-null  object 
 3   medical_terms_list    41967 non-null  object 
dtypes: float64(1), object(3)
memory usage: 1.6+ MB


In [8]:
import numpy as np
print("Discharge diagnosis - ", diagnosis_icd9category['discharge diagnosis:'][2])
print("Max text length - ", np.max([len(i) for i in diagnosis_icd9category['discharge diagnosis:']]))


Discharge diagnosis -  discharge diagnosis:  copd, coronary artery disease/atypical angina (lad 30%, rca 30%,  ef 63%), hypercholesterolemia, hypothyroidism, hypertension,  hiatal hernia, cerebral vascular accident,s/p motor vehicle  colision-> head injury & rib fracture.  tbm- s/p tracheoplasty.      discharge condition:  good    discharge instructions:  please update dr.<name> <telephone> office for:  fever,  shortness of breath, chest pain , productive cough or if you  have any questions or concerns.      completed by: <date> 
Max text length -  16087


In [9]:
import re
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')).union(set(string.punctuation)) # stopwords + punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
# preproceesing text
def preprocess(text):

  text = re.sub("discharge diagnosis:" , ' ', text)

  text = re.sub("\[\*\*.*?\*\*\]"," ",text)

  words=word_tokenize(text)
  filtered_sentence = [] 
  # remove stopwords
  for word in words: 
    if word not in stop_words: 
        filtered_sentence.append(word) 
  text = ' '.join(filtered_sentence)
  # lemmatize
  # lemma_word = []
  # wordnet_lemmatizer = WordNetLemmatizer()
  # for w in filtered_sentence:
  #   word1 = wordnet_lemmatizer.lemmatize(w, pos = "n")
  #   word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
  #   word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
  #   lemma_word.append(word3)
  return text



In [11]:
diagnosis_icd9category['preprocessed'] = diagnosis_icd9category['discharge diagnosis:'].apply(lambda x: preprocess(x))

In [12]:
diagnosis_icd9category

,HADM_ID,discharge diagnosis:,icd9_category,medical_terms_list,preprocessed
2,167118.0,"discharge diagnosis: copd, coronary artery di...","[496, 553, 518, 453, 519, 276]","[coronary artery disease/atypical angina, hype...",copd coronary artery disease/atypical angina l...
3,196489.0,discharge diagnosis: primary: 1. chronic obs...,"[487, 416, 272, 530, 414, 553, 531, 518, 401, ...","[chronic obstructive pulmonary disease, respir...",primary 1. chronic obstructive pulmonary disea...
4,135453.0,discharge diagnosis: 1. cervical spondylosis ...,"[V12, 788, 805, 507, V45, 401, 427, 453, E88, ...","[spondylosis, calcification of posterior, frac...",1. cervical spondylosis calcification posterio...
5,170490.0,discharge diagnosis: brain lesion discharg...,"[710, 225, 443, 714, 530, V43]","[infection, pain, pain, constipation, docusate...",brain lesion discharge condition mental status...
6,134727.0,discharge diagnosis: pneumonia discharge c...,"[486, 518, V45, V58, 401, 577, 415, 428]",[],pneumonia discharge condition expired discharg...
...,...,...,...,...,...
59606,127716.0,discharge diagnosis: aortic stenosis and coro...,"[416, 424, 272, V10, V85, 414, 250, 401, 278, ...","[coronary artery disease, erythema, edema, red...",aortic stenosis coronary artery disease s/p ao...
59609,186377.0,discharge diagnosis: coronary artery disease ...,"[424, 511, 305, 707, 403, 496, 784, 250, 414, ...","[diabetes, diastolic heart failure, erythema, ...",coronary artery disease pmh hypertension diabe...
59615,156564.0,discharge diagnosis: left knee infection chr...,"[V10, 403, 584, V43, 428, 285, 496, 250, V58, ...","[obstructive pulmonary disease, kidney injury,...",left knee infection chronic obstructive pulmon...
59631,172559.0,discharge diagnosis: primary: small bowel ob...,"[197, 507, 185, 427, 261, 585, V66, 560, 783, ...","[bowel obstruction, prostate cancer, diverticu...",primary small bowel obstruction metastatic pro...


In [13]:
len(diagnosis_icd9category)

41967

In [14]:
batch_1 = diagnosis_icd9category[:2000]

In [15]:
print("Max text length - ", np.max([len(i) for i in batch_1['preprocessed']]))

Max text length -  6313


In [16]:
batch_1['preprocessed'] = batch_1['preprocessed'].apply(lambda x: x[:256])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
print("Max text length - ", np.max([len(i) for i in batch_1['preprocessed']]))

Max text length -  256


In [18]:
#DistillBERT
# model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# BERT instead of distilBERT
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [19]:
# tokenize the sentences and vectorize it 
tokenized = batch_1['preprocessed'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
tokenized

2       [101, 8872, 2094, 21887, 2854, 16749, 4295, 10...
3       [101, 3078, 1015, 1012, 11888, 27885, 3367, 68...
4       [101, 1015, 1012, 28711, 11867, 15422, 8516, 1...
5       [101, 4167, 4649, 3258, 11889, 4650, 5177, 357...
6       [101, 18583, 11889, 4650, 13735, 11889, 8128, ...
                              ...                        
2644    [101, 1048, 22432, 2019, 11236, 7274, 2213, 14...
2645    [101, 18634, 8761, 14671, 11463, 15909, 2271, ...
2646    [101, 3674, 19686, 6441, 1017, 7382, 1054, 119...
2648    [101, 1015, 1012, 14092, 2026, 24755, 25070, 1...
2649    [101, 2482, 4140, 3593, 26261, 27109, 1055, 10...
Name: preprocessed, Length: 2000, dtype: object

In [21]:
len(tokenized[2])

67

In [22]:
# pad all tokens to the same size, so we can represent the input as one 2-d array
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [23]:
max_len

101

In [24]:
np.array(padded).shape

(2000, 101)

In [25]:
padded

array([[  101,  8872,  2094, ...,     0,     0,     0],
       [  101,  3078,  1015, ...,     0,     0,     0],
       [  101,  1015,  1012, ...,     0,     0,     0],
       ...,
       [  101,  3674, 19686, ...,     0,     0,     0],
       [  101,  1015,  1012, ...,     0,     0,     0],
       [  101,  2482,  4140, ...,     0,     0,     0]])

In [26]:
# Attention_mask is variable to tell the model to ignore (mask) the padding we've added when it's processing its input. 
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 101)

In [27]:
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [28]:
# The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [29]:
features = last_hidden_states[0][:,0,:].numpy()

In [30]:
features.shape

(2000, 768)

In [31]:
# encodeing one hot vector for multilabel classification
mlb = MultiLabelBinarizer(['401','427','276','414','272','250','428','518','285','584'])

In [32]:
labels = batch_1['icd9_category']

In [33]:
labels = mlb.fit_transform(labels)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['008', '009', '011', '021', '027', '031', '033', '034', '036', '038', '039', '040', '041', '042', '048', '049', '053', '054', '057', '070', '075', '078', '079', '088', '091', '093', '097', '110', '111', '112', '117', '123', '135', '136', '150', '151', '152', '153', '154', '155', '156', '157', '158', '161', '162', '164', '172', '173', '174', '175', '176', '183', '185', '188', '189', '191', '192', '193', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '209', '211', '212', '214', '215', '216', '218', '220', '225', '227', '228', '233', '235', '237', '238', '239', '240', '241', '242', '244', '245', '246', '249', '251', '252', '253', '255', '256', '257', '259', '261', '263', '265', '266', '268', '270', '271', '273', '274', '275', '277', '278', '279', '280', '281', '282', '283', '284', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '29

In [34]:
labels.shape

(2000, 10)

In [35]:
labels

array([[0, 0, 1, ..., 1, 0, 0],
       [1, 0, 1, ..., 1, 0, 1],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [36]:
# use the train features from BERT model to add another layer for classfication.

train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [37]:
from sklearn.multiclass import OneVsRestClassifier

In [38]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf = OneVsRestClassifier(lr_clf)
lr_clf.fit(train_features,train_labels)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='liblinear', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [39]:
predicted = lr_clf.predict(test_features)

In [40]:
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import classification_report

In [41]:
print("Hamming loss for Logistic Regression:")
print(hamming_loss(test_labels,predicted))
print(precision_score(test_labels,predicted,average='micro'))
print(recall_score(test_labels,predicted,average='micro'))
print(f1_score(test_labels,predicted,average='micro'))

Hamming loss for Logistic Regression:
0.3244
0.5277777777777778
0.425206124852768
0.4709719504240052
